In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [ ]:
# data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/CRC_survival_data.csv', index_col=0, sep='\t')

In [ ]:
# read in tsv file COADREAD_TCGA_data.tsv
data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/COADREAD_TCGA_data.tsv', sep='\t', index_col=0)


In [ ]:
data

In [ ]:
clusters_data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/quantile_patient_clusters_0.2_cluster4.csv', index_col=0, sep=',')
clusters_data['Patients'] = clusters_data.index.str.replace('_bootstrap', '').str.replace('.', '-')


In [ ]:

# Filter to retain only overlapping samples
overlapping_samples = set(clusters_data['Patients']).intersection(data['samples'])
filtered_clusters_data = clusters_data[clusters_data['Patients'].isin(overlapping_samples)]
filtered_survival_data = data[data['samples'].isin(overlapping_samples)]

# Check the filtered data
filtered_survival_data

In [ ]:
# Merge the two datasets on the sample identifiers
merged_data = pd.merge(
    filtered_clusters_data,
    filtered_survival_data,
    left_on='Patients',
    right_on='samples'
)
# rank by Cluster column
merged_data = merged_data.sort_values(by='Cluster')

merged_data

In [ ]:
# change the cluster column to non-numeric
merged_data['Cluster'] = merged_data['Cluster'].astype(str)

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
import numpy as np

# Create a months column
merged_data['OS.time_months'] = merged_data['OS.time'] / 30

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Identify clusters and ensure '3' is one of them
clusters = merged_data['Cluster'].unique()

fig, ax = plt.subplots(figsize=(7, 4))

ax.grid(False)  # No grid lines

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Black spines and ticks
for spine in ax.spines.values():
    spine.set_color('black')
    spine.set_linewidth(1.5)
ax.tick_params(axis='x', colors='black', direction='out', length=6, width=1.5)
ax.tick_params(axis='y', colors='black', direction='out', length=6, width=1.5)

# Ensure top and right spines are visible
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)

# Plot survival curves for each cluster using months
for cluster in clusters:
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time_months', 'OS'])
    kmf.fit(durations=cluster_data['OS.time_months'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the overall log-rank test across all clusters
results_all = multivariate_logrank_test(
    event_durations=merged_data['OS.time_months'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)
p_value_all = results_all.p_value

# 3 years in months is 36 months
max_months = 80

# Annotate the global p-value
plt.text(
    x=max_months * 0.6,
    y=0.1, 
    s=f'Overall Log-Rank p-value = {p_value_all:.3e}',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

# Now compare Cluster 3 to each other cluster individually
if '3' in clusters:
    other_clusters = [c for c in clusters if c != '3']
    y_position = 0.05

    for oc in other_clusters:
        # Subset the data to only Cluster 3 and the other cluster
        subset_mask = merged_data['Cluster'].isin(['3', oc])
        subset_data = merged_data[subset_mask]

        # Perform log-rank test for just these two groups (in months)
        results_pair = multivariate_logrank_test(
            event_durations=subset_data['OS.time_months'],
            event_observed=subset_data['OS'],
            groups=subset_data['Cluster']
        )
        p_value_pair = results_pair.p_value

        plt.text(
            x=max_months * 0.6,
            y=y_position, 
            s=f'Cluster 3 vs Cluster {oc} p-value = {p_value_pair:.3e}',
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

        y_position -= 0.05
else:
    print("Warning: Cluster '3' not found in the data. Cannot do pairwise comparisons.")

# Add plot labels and title
# Set X-axis Interval
plt.xticks(ticks=range(0, max_months + 1, 12))  # Set x-axis interval to every 6 months

plt.title('Survival Analysis of COAD_READ patients by Cluster based on quantile')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (months)')  # Changed label to months
plt.ylabel('Survival Probability')
plt.xlim(0, max_months)  # Limit to 36 months (3 years)
plt.ylim(0.3, 1.0)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
import numpy as np

# Create a months column
merged_data['OS.time_months'] = merged_data['OS.time'] / 30

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Identify clusters and ensure '3' is one of them
clusters = merged_data['Cluster'].unique()

fig, ax = plt.subplots(figsize=(7, 4))

ax.grid(False)  # No grid lines

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Black spines and ticks
for spine in ax.spines.values():
    spine.set_color('black')
    spine.set_linewidth(1.5)
ax.tick_params(axis='x', colors='black', direction='out', length=6, width=1.5)
ax.tick_params(axis='y', colors='black', direction='out', length=6, width=1.5)

# Ensure top and right spines are visible
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)

# Plot survival curves for each cluster using months
for cluster in clusters:
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time_months', 'OS'])
    kmf.fit(durations=cluster_data['OS.time_months'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the overall log-rank test across all clusters
results_all = multivariate_logrank_test(
    event_durations=merged_data['OS.time_months'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)
p_value_all = results_all.p_value

# 3 years in months is 36 months
max_months = 42

# Annotate the global p-value
plt.text(
    x=max_months * 0.6,
    y=0.5, 
    s=f'Overall Log-Rank p-value = {p_value_all:.3e}',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

# Now compare Cluster 3 to each other cluster individually
if '3' in clusters:
    other_clusters = [c for c in clusters if c != '3']
    y_position = 0.45

    for oc in other_clusters:
        # Subset the data to only Cluster 3 and the other cluster
        subset_mask = merged_data['Cluster'].isin(['3', oc])
        subset_data = merged_data[subset_mask]

        # Perform log-rank test for just these two groups (in months)
        results_pair = multivariate_logrank_test(
            event_durations=subset_data['OS.time_months'],
            event_observed=subset_data['OS'],
            groups=subset_data['Cluster']
        )
        p_value_pair = results_pair.p_value

        plt.text(
            x=max_months * 0.6,
            y=y_position, 
            s=f'Cluster 3 vs Cluster {oc} p-value = {p_value_pair:.3e}',
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

        y_position -= 0.05
else:
    print("Warning: Cluster '3' not found in the data. Cannot do pairwise comparisons.")

# Add plot labels and title
# Set X-axis Interval
plt.xticks(ticks=range(0, max_months + 1, 6))  # Set x-axis interval to every 6 months

plt.title('Survival Analysis of COAD_READ patients by Cluster based on quantile')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (months)')  # Changed label to months
plt.ylabel('Survival Probability')
plt.xlim(0, max_months)  # Limit to 36 months (3 years)
plt.ylim(0.65, 1.0)
plt.legend()
plt.grid(True)
plt.show()


# 3 years survival plot

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
import numpy as np

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Identify clusters and ensure '3' is one of them
clusters = merged_data['Cluster'].unique()

fig, ax = plt.subplots(figsize=(8, 6))

ax.grid(False)  # No grid lines

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Black spines and ticks
for spine in ax.spines.values():
    spine.set_color('black')
    spine.set_linewidth(1.5)
ax.tick_params(axis='x', colors='black', direction='out', length=6, width=1.5)
ax.tick_params(axis='y', colors='black', direction='out', length=6, width=1.5)

# Ensure top and right spines are visible
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)

# Plot survival curves for each cluster
for cluster in clusters:
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time', 'OS'])
    kmf.fit(durations=cluster_data['OS.time'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the overall log-rank test across all clusters
results_all = multivariate_logrank_test(
    event_durations=merged_data['OS.time'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)
p_value_all = results_all.p_value

# Annotate the global p-value
max_time = merged_data['OS.time'].max()
plt.text(
    x=1093 * 0.6,
    y=0.65, 
    s=f'Overall Log-Rank p-value = {p_value_all:.3e}',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

# Now compare Cluster 3 to each other cluster individually
# Assuming cluster '3' exists in the data
if '3' in clusters:
    other_clusters = [c for c in clusters if c != '3']
    # We'll space these annotations below the overall p-value annotation
    y_position = 0.6

    for oc in other_clusters:
        # Subset the data to only Cluster 3 and the other cluster
        subset_mask = merged_data['Cluster'].isin(['3', oc])
        subset_data = merged_data[subset_mask]

        # Perform log-rank test for just these two groups
        results_pair = multivariate_logrank_test(
            event_durations=subset_data['OS.time'],
            event_observed=subset_data['OS'],
            groups=subset_data['Cluster']
        )
        p_value_pair = results_pair.p_value

        # Annotate the pairwise p-value
        plt.text(
            x=1093 * 0.6,
            y=y_position, 
            s=f'Cluster 3 vs Cluster {oc} p-value = {p_value_pair:.3e}',
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

        y_position -= 0.02  # Move down for the next annotation
else:
    print("Warning: Cluster '3' not found in the data. Cannot do pairwise comparisons.")

# Add plot labels and title
plt.title('Survival Analysis (3 years) of COAD_READ patients by Cluster based on quantile')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (days)')
plt.ylabel('Survival Probability')
plt.xlim(0, 1093)  # Limit to 3 years
plt.ylim(0.7, 1.0)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import multivariate_logrank_test
import numpy as np
import pandas as pd

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Identify clusters and ensure '3' is one of them
clusters = merged_data['Cluster'].unique()

fig, ax = plt.subplots(figsize=(8, 6))

ax.grid(False)  # No grid lines

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Black spines and ticks
for spine in ax.spines.values():
    spine.set_color('black')
    spine.set_linewidth(1.5)
ax.tick_params(axis='x', colors='black', direction='out', length=6, width=1.5)
ax.tick_params(axis='y', colors='black', direction='out', length=6, width=1.5)

# Ensure top and right spines are visible
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)

# Plot survival curves for each cluster
for cluster in clusters:
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time', 'OS'])
    kmf.fit(durations=cluster_data['OS.time'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the overall log-rank test across all clusters
results_all = multivariate_logrank_test(
    event_durations=merged_data['OS.time'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)
p_value_all = results_all.p_value

# Annotate the global p-value
max_time = merged_data['OS.time'].max()
plt.text(
    x=1093 * 0.6,
    y=0.65,
    s=f'Overall Log-Rank p-value = {p_value_all:.3e}',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

# Now compare Cluster 3 to each other cluster individually
# Assuming cluster '3' exists in the data
if '3' in clusters:
    other_clusters = [c for c in clusters if c != '3']
    # We'll space these annotations below the overall p-value annotation
    y_position = 0.6

    for oc in other_clusters:
        # Subset the data to only Cluster 3 and the other cluster
        subset_mask = merged_data['Cluster'].isin(['3', oc])
        subset_data = merged_data[subset_mask]

        # Perform log-rank test for just these two groups
        results_pair = multivariate_logrank_test(
            event_durations=subset_data['OS.time'],
            event_observed=subset_data['OS'],
            groups=subset_data['Cluster']
        )
        p_value_pair = results_pair.p_value

        # Calculate hazard ratios using Cox Proportional Hazards model
        cph_data = subset_data.copy()
        cph_data['Cluster_binary'] = (cph_data['Cluster'] == '3').astype(int)  # Binary variable: 1 for Cluster 3
        cph = CoxPHFitter()
        cph.fit(cph_data[['OS.time', 'OS', 'Cluster_binary']], duration_col='OS.time', event_col='OS')
        hr = cph.summary.loc['Cluster_binary', 'exp(coef)']
        ci_lower = cph.summary.loc['Cluster_binary', 'exp(coef) lower 95%']
        ci_upper = cph.summary.loc['Cluster_binary', 'exp(coef) upper 95%']

        # Annotate the pairwise HR and p-value
        plt.text(
            x=1093 * 0.6,
            y=y_position,
            s=(
                f'Cluster 3 vs Cluster {oc}:\n'
                f'HR = {hr:.2f} (95% CI: {ci_lower:.2f}-{ci_upper:.2f})\n'
                f'p-value = {p_value_pair:.3e}'
            ),
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

        y_position -= 0.06  # Move down for the next annotation
else:
    print("Warning: Cluster '3' not found in the data. Cannot do pairwise comparisons.")

# Add plot labels and title
plt.title('Survival Analysis (3 years) of COAD_READ patients by Cluster based on quantile')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (days)')
plt.ylabel('Survival Probability')
plt.xlim(0, 1093)  # Limit to 3 years
plt.ylim(0.7, 1.0)
plt.legend()
plt.grid(True)
plt.show()


# five years survival plot

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
import numpy as np

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Identify clusters and ensure '3' is one of them
clusters = merged_data['Cluster'].unique()

fig, ax = plt.subplots(figsize=(8, 6))

ax.grid(False)  # No grid lines

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

# Black spines and ticks
for spine in ax.spines.values():
    spine.set_color('black')
    spine.set_linewidth(1.5)
ax.tick_params(axis='x', colors='black', direction='out', length=6, width=1.5)
ax.tick_params(axis='y', colors='black', direction='out', length=6, width=1.5)

# Ensure top and right spines are visible
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)

# Plot survival curves for each cluster
for cluster in clusters:
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time', 'OS'])
    kmf.fit(durations=cluster_data['OS.time'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the overall log-rank test across all clusters
results_all = multivariate_logrank_test(
    event_durations=merged_data['OS.time'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)
p_value_all = results_all.p_value

# Annotate the global p-value
max_time = merged_data['OS.time'].max()
plt.text(
    x=1825 * 0.6,
    y=0.35, 
    s=f'Overall Log-Rank p-value = {p_value_all:.3e}',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

# Now compare Cluster 3 to each other cluster individually
# Assuming cluster '3' exists in the data
if '3' in clusters:
    other_clusters = [c for c in clusters if c != '3']
    # We'll space these annotations below the overall p-value annotation
    y_position = 0.3

    for oc in other_clusters:
        # Subset the data to only Cluster 3 and the other cluster
        subset_mask = merged_data['Cluster'].isin(['3', oc])
        subset_data = merged_data[subset_mask]

        # Perform log-rank test for just these two groups
        results_pair = multivariate_logrank_test(
            event_durations=subset_data['OS.time'],
            event_observed=subset_data['OS'],
            groups=subset_data['Cluster']
        )
        p_value_pair = results_pair.p_value

        # Annotate the pairwise p-value
        plt.text(
            x=1825 * 0.6,
            y=y_position, 
            s=f'Cluster 3 vs Cluster {oc} p-value = {p_value_pair:.3e}',
            fontsize=12,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
        )

        y_position -= 0.02  # Move down for the next annotation
else:
    print("Warning: Cluster '3' not found in the data. Cannot do pairwise comparisons.")

# Add plot labels and title
plt.title('Survival Analysis (5 years) of COAD_READ patients by Cluster based on quantile')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (days)')
plt.ylabel('Survival Probability')
plt.xlim(0, 1825)  # Limit to 3 years
plt.ylim(0.45, 1.0)
plt.legend()
plt.grid(True)
plt.show()


# overall view

In [ ]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test

# Initialize Kaplan-Meier Fitter
kmf = KaplanMeierFitter()

# Prepare the plot
plt.figure(figsize=(10, 7))

# Plot survival curves for each cluster
for cluster in merged_data['Cluster'].unique():
    cluster_data = merged_data[merged_data['Cluster'] == cluster].dropna(subset=['OS.time', 'OS'])
    kmf.fit(durations=cluster_data['OS.time'], event_observed=cluster_data['OS'], label=f'Cluster {cluster}')
    kmf.plot_survival_function(ci_show=False, linestyle='-', linewidth=2)

# Perform the log-rank test across all clusters
results = multivariate_logrank_test(
    event_durations=merged_data['OS.time'],
    event_observed=merged_data['OS'],
    groups=merged_data['Cluster']
)

# Extract and display the p-value
p_value = results.p_value
plt.text(
    x=1095 * 0.6,  # Position on x-axis
    y=0.65,
    s=f'Log-Rank p-value = {p_value:.3e}',  # Format p-value
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.7, edgecolor='black')
)

fig.patch.set_facecolor('white')
ax.set_facecolor('white')


# Add plot labels and title
plt.title('Survival Analysis by Cluster ()')
plt.style.use('seaborn-darkgrid')
plt.xlabel('Time (days)')
plt.ylabel('Survival Probability')
plt.grid(False)
plt.show()


# save the at risk table

In [ ]:
import numpy as np
import pandas as pd
from lifelines import KaplanMeierFitter


merged_data = merged_data.dropna(subset=['OS.time', 'OS', 'Cluster'])
merged_data['Cluster'] = merged_data['Cluster'].astype(str)

# Convert OS.time to months
merged_data['OS.time_months'] = merged_data['OS.time'] / 30

clusters = merged_data['Cluster'].unique()

# Define time intervals (5 months)
max_time_months = merged_data['OS.time_months'].max()
time_intervals = np.arange(0, max_time_months + 6, 6)  # intervals at every 5 months

# Prepare at-risk and censored counts storage
at_risk_table = pd.DataFrame(index=time_intervals, columns=[f'Cluster {cl}' for cl in clusters])
censored_table = pd.DataFrame(index=time_intervals, columns=[f'Cluster {cl}' for cl in clusters])

# Compute at-risk and cumulative censored counts for each cluster at specific month intervals
for clust in clusters:
    mask = (merged_data['Cluster'] == clust)
    durations = merged_data.loc[mask, 'OS.time_months']
    events = merged_data.loc[mask, 'OS']

    kmf = KaplanMeierFitter()
    kmf.fit(durations=durations, event_observed=events, label=f'Cluster {clust}')

    event_table_times = kmf.event_table.index
    at_risk_counts = []
    censored_counts = []
    cumulative_censored = 0  # Initialize cumulative censored count

    for t in time_intervals:
        # Find the closest time in the event table
        closest_time = event_table_times[np.abs(event_table_times - t).argmin()]
        at_risk_counts.append(kmf.event_table.loc[closest_time, 'at_risk'])
        
        # Accumulate censored counts
        current_censored = kmf.event_table.loc[closest_time, 'censored']
        cumulative_censored += current_censored
        censored_counts.append(cumulative_censored)

    at_risk_table[f'Cluster {clust}'] = at_risk_counts
    censored_table[f'Cluster {clust}'] = censored_counts

# Save the at-risk and cumulative censored tables to Excel
with pd.ExcelWriter('at_risk_table_quantile_4clusters.xlsx') as writer:
    at_risk_table.to_excel(writer, sheet_name='At Risk', index_label='Time Interval (months)')
    censored_table.to_excel(writer, sheet_name='Cumulative Censored', index_label='Time Interval (months)')

print("At-risk and cumulative censored tables saved to 'at_risk_and_cumulative_censored_table_months.xlsx'")


# use GEM continuous data to directly compare the survival rate

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd

data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/TCGA_COAD_READ_gsva_results.csv', index_col=0, sep=',')


# Transpose the data to have patients as rows and GEMs as features
data_transposed = data.T

# Perform k-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(data_transposed)

# Add cluster labels to the patients
data_transposed['Cluster'] = clusters

data_transposed['Cluster']


In [ ]:
# Draw a clustered heatmap
sns.clustermap(
    data_transposed,
    cmap="viridis",      # Color map
    method="ward",       # Clustering method (hierarchical)
    metric="euclidean",  # Distance metric
    standard_scale=1,    # Normalize each column (GEM)
    figsize=(12, 8),     # Adjust the size of the plot
    row_cluster=True,    # Cluster rows (patients)
    col_cluster=True     # Cluster columns (GEMs)
)

plt.title("Clustered Heatmap of GEM Expression")
plt.show()